In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pyperclip
import time
import pandas as pd
import os
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException

# ✅ 브라우저 창 크기 설정
WINDOW_WIDTH = 1100
WINDOW_HEIGHT = 1000

# ✅ 브라우저 초기화 및 로그인
def setup_browser(position_x):
    driver = webdriver.Chrome()
    driver.set_window_size(WINDOW_WIDTH, WINDOW_HEIGHT)
    driver.set_window_position(position_x, 0)
    driver.get('https://console.thebackend.io/ko/login')
    time.sleep(0.2)
    return driver

# ✅ 로그인 함수
def login_to_console(driver):
    try:
        id_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "username"))
        )
        pyperclip.copy("dksrufp0607@naver.com")
        id_input.send_keys(Keys.CONTROL, 'v')

        pw_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "password"))
        )
        pyperclip.copy("ghkfkd159!")
        pw_input.send_keys(Keys.CONTROL, 'v')

        login_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[type="submit"]'))
        )
        login_button.click()
        print("✅ 로그인 완료!")
    except Exception as e:
        print(f"⚠️ 로그인 실패: {e}")

# ✅ 콘솔 페이지 이동
def navigate_to_page(driver):
    article_url = 'https://console.thebackend.io/ko/project/1ea3f14d34e89530ea88b3245bc82dc17d5f52ce1554049f19fce9219a847cfce18bb8891c9ffe90bc65e2b9a3b981853fc5513c1dd200afc9590ba6bfd5fced4230647d25328849e0917641/baseGameInfo/data'
    driver.get(article_url)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.TAG_NAME, "body"))
    )
    print("✅ 페이지 로드 완료!")

# ✅ "확인" 버튼 클릭
def confirm_click(driver):
    try:
        print("🔍 '확인' 버튼 탐색 중...")
        confirm_button = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "확인")]'))
        )
        confirm_button.click()
        print("✅ 확인 버튼 클릭 완료!")
    except TimeoutException:
        print("⚠️ '확인' 버튼을 찾지 못했습니다. 페이지가 제대로 로드되었는지 확인하세요.")
    except Exception as e:
        print(f"⚠️ 확인 버튼 클릭 실패: {e}")

# ✅ CSV 데이터 로드 함수 (검색용)
def get_csv_value(row, col):
    csv_path = "C:/엑셀/Ranking_1.csv"  # CSV 경로 절대 경로로 설정
    try:
        df = pd.read_csv(csv_path)
        if row < df.shape[0] and col < df.shape[1]:
            print(f"🔹 {row}행 3열 값 (출력용): {df.iloc[row, 2]}")
            return df.iloc[row, col]
        else:
            print("⚠️ CSV 파일의 크기를 초과하는 인덱스입니다.")
            return None
    except FileNotFoundError:
        print(f"⚠️ CSV 파일을 찾을 수 없습니다: {csv_path}")
        return None
    except Exception as e:
        print(f"⚠️ CSV 로드 중 오류 발생: {e}")
        return None

# ✅ 검색 실행 함수
def execute_tasks(driver, row, col):
    try:
        b2_value = get_csv_value(row, col)
        if not b2_value:
            print("⚠️ CSV에서 값을 가져오지 못했습니다. 작업을 중단합니다.")
            return

        table_dropdown = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//div[contains(@class, "_manageInfo-data_1rntr_71")]'))
        )
        table_dropdown.click()
        print("✅ 테이블 드롭다운 클릭 완료!")

        base_data_option = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//div[@role="option"][span[text()="BASE_DATA"]]'))
        )
        base_data_option.click()
        print("✅ BASE_DATA 선택 완료!")

        detail_search_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//p[contains(text(), "상세 검색")]'))
        )
        try:
            detail_search_button.click()
        except ElementClickInterceptedException:
            print("⚠️ '상세 검색' 버튼 클릭 차단됨. JavaScript로 재시도...")
            driver.execute_script("arguments[0].click();", detail_search_button)
        print("✅ 상세 검색 클릭 완료!")

        search_input = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.NAME, "defaultSearchValue"))
        )
        search_input.clear()
        search_input.send_keys(b2_value)
        print(f"✅ B2 값 입력 완료: {b2_value}")

        search_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//button[contains(@class, "ui medium positive button") and @type="submit"]'))
        )
        search_button.click()
        print("✅ 검색 버튼 클릭 완료!")

        uuid_element = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//p[contains(@class, "_link_a1mor_113")]'))
        )
        uuid_element.click()
        print("✅ UUID 클릭 완료!")
    except Exception as e:
        print(f"⚠️ 검색 작업 중 오류 발생: {e}")

# ✅ 엑셀에서 데이터 로드 함수
def load_excel_data(filename):
    try:
        df = pd.read_excel(filename)
        data = df[["Column Name", "Data Type", "Value"]].values.tolist()
        print(f"✅ 엑셀 데이터 로드 완료! 경로: {os.path.abspath(filename)}")
        return data
    except FileNotFoundError:
        print(f"⚠️ 엑셀 파일을 찾을 수 없습니다: {filename}")
        return []
    except Exception as e:
        print(f"⚠️ 엑셀 파일 로드 중 오류 발생: {e}")
        return []

# ✅ 데이터 붙여넣기 함수
def paste_data(driver, data):
    try:
        edit_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//button[contains(@class, "ui mini primary button") and text()="수정"]'))
        )
        edit_button.click()
        print("✅ 수정 버튼 클릭 완료!")

        elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "_columnBorder-edit-modal_1rntr_47"))
        )

        for element, (col_name, data_type, new_value) in zip(elements, data):
            try:
                col_input = element.find_element(By.CSS_SELECTOR, 'input[type="text"][name$=".columnName"]')
                current_col_name = col_input.get_attribute("value")
                if current_col_name != col_name:
                    print(f"⚠️ 컬럼 명 불일치: 예상={col_name}, 실제={current_col_name}")
                    continue

                dropdown = element.find_element(By.CSS_SELECTOR, 'div[name$=".dataType"][role="listbox"]')
                dropdown.click()
                time.sleep(0.5)
                option = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.XPATH, f'//div[@role="option"]/span[text()="{data_type}"]'))
                )
                option.click()
                print(f"✅ {col_name} 데이터 타입 설정 완료: {data_type}")

                try:
                    value_field = element.find_element(By.CSS_SELECTOR, 'textarea[name$=".dataValue"]')
                    value_field.clear()
                    value_field.send_keys(str(new_value))
                    print(f"✅ {col_name} 값 입력 완료: {new_value}")
                except NoSuchElementException:
                    print(f"⚠️ {col_name} 값 필드가 없음 (object 타입일 가능성 있음)")
                    continue

            except NoSuchElementException as e:
                print(f"⚠️ 요소 처리 실패 ({col_name}): {e}")
                continue

        save_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "저장")]'))
        )
        save_button.click()
        print("✅ 저장 완료!")
    except Exception as e:
        print(f"⚠️ 데이터 붙여넣기 실패: {e}")

# ✅ 메인 실행
if __name__ == "__main__":
    # 절대 경로로 엑셀 파일 지정 (사용자 환경에 맞게 수정 필요)
    excel_filename = "C:/Users/jjg19/Documents/scraped_data2.xlsx"  # 본인의 경로로 변경

    # 현재 작업 디렉토리 출력 (디버깅용)
    print(f"🔍 현재 작업 디렉토리: {os.getcwd()}")

    # 파일 존재 여부 확인
    if not os.path.exists(excel_filename):
        print(f"⚠️ 파일이 존재하지 않습니다: {excel_filename}")
        exit()

    # 엑셀에서 데이터 로드
    data = load_excel_data(excel_filename)
    if not data:
        print("⚠️ 데이터 로드 실패로 인해 프로그램을 종료합니다.")
        exit()

    driver1 = setup_browser(0)
    try:
        login_to_console(driver1)
        navigate_to_page(driver1)
        confirm_click(driver1)
        execute_tasks(driver1, 0, 5)  # CSV에서 검색 값 가져오기
        paste_data(driver1, data)    # 엑셀 데이터로 값 붙여넣기
    except KeyboardInterrupt:
        print("⚠️ 사용자가 프로그램을 중단했습니다.")
    finally:
        driver1.quit()
        print("✅ 브라우저가 종료되었습니다.")

🔍 현재 작업 디렉토리: c:\JJG\Start\sjdj
⚠️ 파일이 존재하지 않습니다: C:/Users/jjg19/Documents/scraped_data2.xlsx
⚠️ 엑셀 파일을 찾을 수 없습니다: C:/Users/jjg19/Documents/scraped_data2.xlsx
⚠️ 데이터 로드 실패로 인해 프로그램을 종료합니다.
✅ 로그인 완료!
✅ 페이지 로드 완료!
🔍 '확인' 버튼 탐색 중...
⚠️ '확인' 버튼을 찾지 못했습니다. 페이지가 제대로 로드되었는지 확인하세요.
🔹 0행 3열 값 (출력용): 슈바니
⚠️ 검색 작업 중 오류 발생: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7FE646EE5+28773]
	(No symbol) [0x00007FF7FE5B25D0]
	(No symbol) [0x00007FF7FE448FAA]
	(No symbol) [0x00007FF7FE49F286]
	(No symbol) [0x00007FF7FE49F4BC]
	(No symbol) [0x00007FF7FE4F2A27]
	(No symbol) [0x00007FF7FE4C728F]
	(No symbol) [0x00007FF7FE4EF6F3]
	(No symbol) [0x00007FF7FE4C7023]
	(No symbol) [0x00007FF7FE48FF5E]
	(No symbol) [0x00007FF7FE4911E3]
	GetHandleVerifier [0x00007FF7FE99422D+3490733]
	GetHandleVerifier [0x00007FF7FE9ABA13+3586963]
	GetHandleVerifier [0x00007FF7FE9A144D+3544525]
	GetHandleVerifier [0x00007FF7FE70C9AA+838442]
	(No symbol) [0x00007FF7FE5BD01F]
	(No symbol) [0x00007FF7FE5B95E4]
	(No symbol) [0x000

: 